In [37]:
import pandas as pd
import numpy as np

filename = 'InternationalStandards.csv'


dfdatafile = pd.read_csv(filename, usecols=['Name','ActualIU', 'Target 1 Ct'])

# Assign the data to two dataframes, one (dfstandards) which has all the values of the standard curve and another which has the Ct values of the unknown sample
dfdatafile.rename(columns = {"ActualIU": "IU"}, inplace=True)
dfstandards = dfdatafile.loc[(dfdatafile['Name'].str.contains("VLP") == False)& (dfdatafile['Name'].str.contains("Blank") == False)].copy()
dfstandards['IU'] = dfstandards['IU'].astype('float')
dfstandards['Target 1 Ct'] = dfstandards['Target 1 Ct'].astype('float')
dfvlp = dfdatafile.loc[dfdatafile['Name'].str.contains("VLP")].copy()
dfvlp['Target 1 Ct'] = dfvlp['Target 1 Ct'].astype('float')

print(dfstandards)

     Name  Target 1 Ct       IU
0   95500        21.92  95500.0
1   95500        22.08  95500.0
2   95500        22.09  95500.0
3    9550        25.28   9550.0
4    9550        25.45   9550.0
5    9550        25.40   9550.0
6     995        28.73    995.0
7     995        28.70    995.0
8     995        28.85    995.0
9    99.5        32.09     99.5
10   99.5        31.86     99.5
11   99.5        31.88     99.5


In [38]:
import statsmodels.api as sm

# Fit regression model (using the natural log of one of the regressors)
X = sm.add_constant(np.log10(dfstandards['IU']))
regmodel = sm.OLS(dfstandards['Target 1 Ct'], X).fit()

# Display results of the regression model with the slope, intercept and R-squared
summary = regmodel.summary()
print(summary)


                            OLS Regression Results                            
Dep. Variable:            Target 1 Ct   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 1.400e+04
Date:                Sun, 04 Sep 2022   Prob (F-statistic):           4.55e-17
Time:                        19:35:50   Log-Likelihood:                 10.730
No. Observations:                  12   AIC:                            -17.46
Df Residuals:                      10   BIC:                            -16.49
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         38.6665      0.103    374.643      0.0

/usr/local/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1477: UserWarning:

kurtosistest only valid for n>=20 ... continuing anyway, n=12



In [58]:
import math
import plotly.graph_objects as go

# Specify colour blind safe colours for use in different plots
colors = ['rgba(0,0,0,1)','rgba(230,159,0,1)', 'rgba(86,180,233, 1)', 'rgba(0,158,115,1)', 'rgba(0,114,178,1)', 'rgba(213,94,0, 1)', 'rgba(204,121,167,1)','rgba(0,0,0,1)','rgba(0,0,0,1)']

dfresiduals = dfstandards

# Assign values of the calculated slope and intercept to variables for the equation y = b0 + b1x (SEq 1)
b0 = regmodel.params[0]
b1 = regmodel.params[1]

# Calculate the residuals for each point in the standard curve
dfresiduals['predct'] = np.log10(dfstandards['IU'])*b1 +b0
dfresiduals['residuals'] = dfresiduals['Target 1 Ct'] - dfresiduals['predct']

# Add a new column to the dataframe with logged values of the IU
dfresiduals['logIU'] = np.log10(dfresiduals['IU'])

# Create a scatter plot of the residuals
trace = []
trace.append(go.Scatter(x=dfresiduals['IU'], y=dfresiduals['residuals'], name = 'Ct', mode='markers', marker = dict(color =colors[0])))

layoutresiduals=go.Layout(font={'family':'Roboto'},
                     paper_bgcolor='rgba(255,255,255,1)',
                     plot_bgcolor='rgba(255,255,255,1)',
                     margin=dict(l=100, r=40, t=40, b=100),
                     autosize=False,
                     width=500,
                     height=375,
                     titlefont={'size': 20,
                               'color':'black'},
                     xaxis=dict(title='copies/mL',
                            titlefont = dict(size=18,
                                        color='black'),
                            showticklabels=True,
                            tickangle=0,
                            type="log", range=[1,5.5],
                            tickfont=dict(
                                size=16,
                                color='black'
                                    ),
                            showgrid=False,
                            tickcolor = '#000',
                            tickwidth=1.5,
                            ticks='outside',
                            exponentformat = 'power',
                            tickvals=[10, 100, 1000,10000,100000]
                               ),
                    yaxis=dict(
                        title='Ct',
                        titlefont=dict(
                            size=18,
                            color='black'),
                        showticklabels=True,
                        tickangle=0,
                        tickfont=dict(
                            size=16,
                            color='black'),
                        tickcolor = '#000',
                        zerolinecolor='#000',
                        zerolinewidth=1.5,
                        rangemode='tozero',
                        exponentformat='none',
                        showexponent='all',
                        tickwidth=1.5,
                        ticks='outside',
                        range = [-0.5,0.5]
                            ))

fig= go.Figure(data=trace, layout=layoutresiduals)

fig.update_layout(showlegend=False)
fig.show()

# Write image of residuals to folder, using kaleido
fig.write_image('images/residuals.svg', engine="kaleido",scale=1)



In [40]:
import scipy

# Calculate the number of unknown samples based on the size of the VLP dataframe
numsamples = dfvlp.shape[0]

# Calculate the number of standard curve samples based on the size of the standard curve dataframe
J = dfresiduals.shape[0]

# Calculate the number of degrees of freedom
pointsstdcurve = dfresiduals['IU'].nunique()
numreplicates = (dfresiduals.duplicated(subset=['IU']).sum()+pointsstdcurve)/pointsstdcurve
sumJmin1 = pointsstdcurve*(numreplicates-1)

# Calculate the tscore associated with the degrees of freedom
alpha = 0.05
tscore = abs(scipy.stats.t.ppf(alpha/2, sumJmin1))

# Calculate the efficiency of the PCR reaction
efficiency = 10**(-1/b1)-1

# Calculate the predicted value of x based on the mean Ct values of the unknown samples
ymean = dfvlp['Target 1 Ct'].mean()
xpred = (ymean-b0)/b1

# Calculate the residuals of the unknown samples and square them
dfvlp['residualssqr'] = (dfvlp['Target 1 Ct'] - dfvlp['Target 1 Ct'].mean())**2

# Sum the squared residuals of the unknown samples
vlpsumresid = dfvlp['residualssqr'].sum()

# Calculate the mean of the standards for each point on the standard curve
dfmeanstandards = dfstandards.groupby(['IU'], as_index=False).mean()[['IU', 'Target 1 Ct']]
dfmeanstandards.rename(columns={'Target 1 Ct':'mean ct'}, inplace=True)
# Merge the dataframes to add the calculated mean ct column to the dfstandards dataframe
dfstandardscalc = pd.merge(dfstandards,dfmeanstandards,on = "IU")
# Calculate the residuals of the standard curve based on the mean ct values of each point on the standard curve
dfstandardscalc['residuals'] = dfstandardscalc['Target 1 Ct'] - dfstandardscalc['mean ct']
# Square the residuals of the standard curve
dfstandardscalc['calcsqr'] = dfstandardscalc['residuals']**2

# calculate the s^2p value (SEq 4)
ssqrp = (dfstandardscalc['calcsqr'].sum())/(sumJmin1) 

# Calculate x double bar (SEq 6)
xdblbar = (dfstandardscalc['logIU'].sum())/J
dfstandardscalc['xi'] = (dfstandardscalc['logIU']-xdblbar)**2

# Calculate s_xx (SEq 5)
sxx = dfstandardscalc['xi'].sum()

# Calculate the value of g (SEq 11)
g=(ssqrp*tscore**2)/(sxx*b1**2)

# Calculate the upper and lower confidence limits of log of the unknown samples (SEq 12)
xlower = xpred + (((xpred-xdblbar)*g+(math.sqrt(ssqrp)*tscore/b1))*(((xpred-xdblbar)**2/sxx)+(1-g)*(1/numsamples+1/J))**0.5)/(1-g)
xupper = xpred + (((xpred-xdblbar)*g-(math.sqrt(ssqrp)*tscore/b1))*(((xpred-xdblbar)**2/sxx)+(1-g)*(1/numsamples+1/J))**0.5)/(1-g)

# Calculate the upper and lower confidence limits of the ct values of the unknown samples (SEq 13)
ylower = ymean - tscore*math.sqrt(ssqrp)*((xpred-xdblbar)**2/sxx+1/numsamples+1/J)**0.5
yupper = ymean + tscore*math.sqrt(ssqrp)*((xpred-xdblbar)**2/sxx+1/numsamples+1/J)**0.5

# Calculate the intercepts of the prediction interval lines that run parallel to the calculated regression line
interceptlower = ymean -b1*xlower
interceptupper = ymean -b1*xupper

# print out all calculated values
print('slope:' + str(b1))
print('intercept:' + str(b0))
print('efficiency:' + str(efficiency))
print('xpred: '+str(10**xpred))
print('ymean: '+str(ymean))
print('dof: '+str(sumJmin1))
print('number of unknown sample replicates: '+str(numsamples))
print('s2p: '+str(ssqrp))
print('xdblbar: '+str(xdblbar))
print('sxx: '+str(sxx))
print('g: '+str(g))
print('xupper: '+str(10**xupper))
print('xlower: '+str(10**xlower))
print('ylower: '+str(ylower))
print('yupper: '+str(yupper))
print('interceptlower: '+str(interceptlower))
print('interceptupper: '+str(interceptupper))

slope:-3.335984211745958
intercept:38.66645943863722
efficiency:0.9941673919755294
xpred: 831.9851413821831
ymean: 28.924999999999997
dof: 8.0
number of unknown sample replicates: 4
s2p: 0.00981666666666674
xdblbar: 3.4889132261647355
sxx: 14.787116116160096
g: 0.00031721429593360836
xupper: 913.763660990488
xlower: 757.1503874590653
ylower: 28.78882848628346
yupper: 29.061171513716534
interceptlower: 38.529906191564486
interceptupper: 38.80229508670102


In [62]:
# generate predicted values for the calculated regression line
predictions = regmodel.get_prediction()
df_predictions = predictions.summary_frame(alpha=0.05)
df_predictions['IU'] = dfstandards['IU']

# generate predicted values for the prediction interval lines
yvalues = np.arange(1,41)
errorpredminx = (yvalues - interceptlower)/b1
errorpredmaxx = (yvalues - interceptupper)/b1

# plot values of the prediction intervals of the concentration of the unknown samples

layout=go.Layout(font={'family':'Arial'},
                     paper_bgcolor='rgba(255,255,255,1)',
                     plot_bgcolor='rgba(255,255,255,1)',
                     margin=dict(l=80, r=40, t=40, b=80),
                     autosize=False,
                     width=400,
                     height=350,
                     titlefont={'size': 20,
                               'color':'black'},
                     xaxis=dict(title='copies/mL',
                            titlefont = dict(size=16,
                                        color='black'),
                            showticklabels=True,
                            tickangle=0,
                        zerolinecolor='#000',
                        zerolinewidth=2,
                            # tickprefix = "<b>",
                            # ticksuffix = "</b>",
                            type="log", range=[1,5.5],
                            tickfont=dict(
                                size=12,
                                color='black'
                                    ),
                            showgrid=False,
                            tickcolor = '#000',
                            tickwidth=1.5,
                            ticks='outside',
                            exponentformat = 'power',
                            tickvals=[10, 100, 1000,10000,100000],
                               ),
                    yaxis=dict(
                        title='Ct',
                        titlefont=dict(
                            size=16,
                            color='black'),
                        showticklabels=True,
                        tickangle=0,
                        tickfont=dict(
                            size=12,
                            color='black'),
                        tickcolor = '#000',
                        zerolinecolor='#000',
                        zerolinewidth=2,
                        # tickprefix = "<b>",
                        # ticksuffix = "</b>",
                        rangemode='tozero',
                        exponentformat='none',
                        showexponent='all',
                        tickwidth=1.5,
                        ticks='outside',
                        range = [0,40],
                           )
    )



trace = []

# plot of the prediction intervals below and above the calculated regression line

errorpredupper = go.Scatter(
        name='Upper Bound',
        x=10**errorpredmaxx,
        y=yvalues,
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False)

errorpredlower = go.Scatter(
        name='Lower Bound',
        x=10**errorpredminx,
        y=yvalues,
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(230,159,0,0.3)',
        fill='tonexty',
        showlegend=False
    )

# plot of the individual standard curve samples
trace.append(go.Scatter(x=dfstandards['IU'], y=dfstandards['Target 1 Ct'], name = 'Ct', mode='markers', marker = dict(color =colors[0])))
# plot of the calculated regression line
trace.append(go.Scatter(x=df_predictions['IU'], y=df_predictions['mean'], name = 'lowerPI', mode='lines', line = dict(color =colors[0],width=2)))
# addition of vertical lines for the xupper and xlower values and their intercept with the horizontal line from ymean
trace.append(go.Scatter(x=[10**xupper,10**xupper], y=[ymean,0], name = 'PI', mode='lines', line = dict(color =colors[0],width=2,dash='dot')))
trace.append(go.Scatter(x=[10**xlower,10**xlower], y=[ymean,0], name = 'PI', mode='lines', line = dict(color =colors[0],width=2,dash='dot')))
trace.append(go.Scatter(x=[10**0,10**xupper], y=[ymean,ymean], name = 'PI', mode='lines', line = dict(color =colors[0],width=2,dash='dot')))


fig= go.Figure(data=trace, layout=layout)

fig.update_layout(showlegend=False)
fig.show()

fig.write_image('images/std_curve.svg', engine="kaleido",scale=1)


In [63]:
# zoomed in graph of the prediction intervals and the xupper and xlower values and their intercept with the horizontal line from ymean

trace.append(errorpredupper)
trace.append(errorpredlower)


fig= go.Figure(data=trace, layout=layout)
fig.update_layout(showlegend=False)


fig.update_layout(yaxis=dict(
        range=[28, 30],
        tickvals=[28, 29, 30],
    ),
    xaxis=dict(
        range=[2.8, 3.05],
        tickvals=[600, 700, 800, 900,1000,1100]
    ))

fig.show()
fig.write_image('images/std_curve_zoom.svg', engine="kaleido",scale=1)


In [64]:
trace = []

# plot of the individual standard curve samples
trace.append(go.Scatter(x=dfstandards['IU'], y=dfstandards['Target 1 Ct'], name = 'Ct', mode='markers', marker = dict(color =colors[0])))
# plot of the calculated regression line
trace.append(go.Scatter(x=df_predictions['IU'], y=df_predictions['mean'], name = 'lowerPI', mode='lines', line = dict(color =colors[0],width=2)))

# plot of the yupper, ylower and xupper and xlower values and their corresponding intercepts
trace.append(go.Scatter(x=[10**xupper,10**xupper], y=[ylower,0], name = 'PI', mode='lines', line = dict(color =colors[0],dash = 'dash',width=2)))
trace.append(go.Scatter(x=[10**xlower,10**xlower], y=[yupper,0], name = 'PI', mode='lines', line = dict(color =colors[0],dash = 'dash',width=2)))
trace.append(go.Scatter(x=[10**0,10**xupper], y=[ylower,ylower], name = 'PI', mode='lines', line = dict(color =colors[0],dash = 'dash',width=2)))
trace.append(go.Scatter(x=[10**0,10**xlower], y=[yupper,yupper], name = 'PI', mode='lines', line = dict(color =colors[0],dash = 'dash',width=2)))



fig= go.Figure(data=trace, layout=layout)

fig.update_layout(showlegend=False)
fig.show()

fig.write_image('images/ypred.svg', engine="kaleido",scale=1)


In [65]:
# zoomed in graph of the prediction intervals of the yupper, ylower, xupper and xlower values and their corresponding intercepts

fig= go.Figure(data=trace, layout=layout)
fig.update_layout(showlegend=False)


fig.update_layout(yaxis=dict(
        range=[28, 30],
        tickvals=[28, 29, 30],
    ),
    xaxis=dict(
        range=[2.8, 3.05],
        tickvals=[600, 700, 800, 900,1000,1100]
    ))

fig.show()
fig.write_image('images/ypred_zoom.svg', engine="kaleido",scale=1)